In [1764]:
FORM_MAPS = {
    "le génitif": "genitive",
    "gen.": "genitive",
    "gén.": "genitive",
    "gén": "genitive",
    "dat.": "dative",
    "dat": "dative",
    "plur.": "plural",
    "plur": "plural",
    "futur": "future",
    "fut.": "future",
    "participe": "participle",
    "comp.": "comparative",
    "comp": "comparative",
    "compar.": "comparative",
    "compar": "comparative",
    "impér.": "imperative",
    "impér": "imperative",
    "diminutif": "diminutive",
    "vocatif": "vocative",
    "gén. fém.": "genitive_feminine",
    "superl.": "superlative"
}

In [1852]:
import json
with open("sjoestedt_phonetique.json") as inf:
    DATA = json.load(inf)

In [424]:
section = 30

if not section in DATA:
    DATA[section] = []

In [384]:
DATA[section] = []

In [1765]:
import re
REGEX_CASE = rf"^([^(]+) \(([^)]+)\) « ([^»]+) », ({'|'.join(FORM_MAPS.keys())}) ([^(]+) \(([^)]+)\)$"
REGEX_CASE_OF = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) « ([^»]+) »$"
REGEX_CASE_OF_BOTH = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_BASE = r"^([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_JUST_FRENCH = r"^([^(]+) « ([^»]+) »$"
REGEX_FROM = r"^([^(]+) \(([^)]+)\) « ([^»]+) »,? de ([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_IN = r"^([^(]+) \(([^)]+)\), dans ([^«]+) « ([^»]+) »$"
REGEX_OR = r"^([^(]+) \(([^)]+)\) ou ([^(]+) \(([^)]+)\) « ([^»]+) »$"

def extract(text, splitter, counter):
    text = text.replace("\u00ad", "").replace("\n", " ")
    for p in text.split(splitter):
            m = re.match(REGEX_BASE, p.strip())
            mjf = re.match(REGEX_JUST_FRENCH, p.strip())
            mf = re.match(REGEX_FROM, p.strip())
            min = re.match(REGEX_IN, p.strip())
            mc = re.match(REGEX_CASE, p.strip())
            mcofb = re.match(REGEX_CASE_OF_BOTH, p.strip())
            mcof = re.match(REGEX_CASE_OF, p.strip())
            mor = re.match(REGEX_OR, p.strip())
            if m:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": m.group(1),
                    "irish": m.group(2),
                    "french": m.group(3),
                })
                counter += 1
            elif mjf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mjf.group(1),
                    "french": mjf.group(2),
                })
                counter += 1
            elif mf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(1),
                    "irish": mf.group(2),
                    "french": mf.group(3).replace("\xad", ""),
                    "from": mf.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(4),
                    "irish": mf.group(5),
                    "french": mf.group(6).replace("\xad", ""),
                })
                counter += 1
            elif mor:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(1),
                    "irish": mor.group(2),
                    "french": mor.group(5),
                    "alt": mor.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(3),
                    "irish": mor.group(4),
                    "french": mor.group(5),
                    "alt": mor.group(2)
                })
                counter += 1
            elif min:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(1),
                    "irish": min.group(2),
                    "in": min.group(3)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(3),
                    "french": min.group(4),
                })
                counter += 1
            elif mc:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(1),
                    "irish": mc.group(2),
                    "french": mc.group(3),
                    FORM_MAPS[mc.group(4)]: mc.group(6)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(5),
                    "irish": mc.group(6),
                    f"{FORM_MAPS[mc.group(4)]}_of": mc.group(2)
                })
                counter += 1
            elif mcofb:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcofb.group(1),
                    "irish": mcofb.group(2),
                    f"{FORM_MAPS[mcofb.group(3)]}_of": mcofb.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "irish": mcofb.group(5),
                    "transcription": mcofb.group(4),
                    "french": mcofb.group(6),
                    FORM_MAPS[mcofb.group(3)]: mcofb.group(2)
                })
                counter += 1
            elif mcof:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(1),
                    "irish": mcof.group(2),
                    f"{FORM_MAPS[mcof.group(3)]}_of": mcof.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(4),
                    "french": mcof.group(5),
                    FORM_MAPS[mcof.group(3)]: mcof.group(2)
                })
                counter += 1
            else:
                print(p)
    print(counter)


In [1932]:
REST = {
    244: "bɔləg (bolg) « ventre » ; marʹɩgʹ (mairg) « pitié, malheur » ; fɔrəməd (formad) « envie » ; bɑləv (balbh) « bègue, muet » ; fɔnəvər (fonnmhar) « désireux » ; dʌrəχə (dorcha) « sombre »",
    246: "ɑχᵊrən (achrann) « difficulté » ; ə χrɑ: (do chrádh) « contrarier »",
    241: "lʲo:rgo:hɩnʹtʹ (leor­dhóthain) « quantité largement suffi­sante »"
}

ɑχᵊrən (achrann) « difficulté », mais ə χrɑ: (do chrádh) « contrarier ».

h, suivi de liquide (cf. §§ 215 et 231) :

fʷɪ:hɩrʹəχ (faoithireach) « moquerie » ; αhɩrʹɩʃ (aithris) « imitation », mais də hrʹi·əl (do thriall) « le but de ton voyage ». La voyelle furtive est parti­culière­ment développée.

Nasale suivie de liquide (cf. §§ 215 et 226) :

ʃaᴜmᵊrə (seomra) « chambre », mais mraχtɩnʹtʹ (mrach­taint) « vivre ».

§ 247. Occlusive suivie de nasale (cf. § 215, 2º ; § 229, 2º) ; dans ce cas et dans ceux qui suivent, la voyelle furtive est plus déve­loppée que devant liquide : αgʹɩnʹɩ (aigne) « esprit », mais gnʹi:ᵊv (gníomh) « action ».

h suivi de nasale (cf. §§ 215 et 231) : fʷɪhɩnʲəχ (foithi­neach) « abrité ».

Nasale suivie de nasale (cf. §§ 215, 2º et 226) :

imʹɩnʹi: (imshníomh) « souci », mais mnɪ: (mnaoi), dat. de bʹαn (bean) « femme ».

Le cas de nm (nʹmʹ), groupe qui ne se rencontre pas en position initiale, diffère du cas de mn (mʹnʹ), et a été mentionné § 244.

§ 248. Occlusive suivie de spirante sonore (cf. §§ 215, 4º et 229, 3º) : ɑdᵊvʷɪ:mʹ (ad­mhuighim) « j’avoue », mais gvɑ:ⁱlʹtʹ (gabháilt) « prendre » (cf. § 279).

§ 249. Les groupes dont il s’agit ont donc une double pronon­ciation, selon qu’ils sont initiaux (explosifs) ou médians (implosivo-explosifs). Il semble cependant qu’il y ait tendance à prononcer les groupes médians comme s’ils étaient initiaux (sans voyelle furtive, quoique avec une explosion audible), là où ces groupes sont suivis de voyelle longue, la voyelle précé­dente étant brève, c’est-à-dire là où l’accent frappe la syllabe suivante. Ainsi : əbrɑ̃:n (Abrán) « Avril », comme brɑt (brat) « châle », non comme ɑbᵊrɩmʹ ; de même əblɑ:ⁱlʹ (abláil) « faire un travail léger, bricoler » ; əprᴜ:n (aprún) « tablier » ; lʹαdrɑ̃:nəχ (leadránach) « paresseux, trainard », comme drαdʹ (draid) « fait de montrer les dents », non comme αdᵊrɩnʹ (eadrainn) « entre nous » : poklʹe:mʹ (pocléim) « saut », comme klʹe:rʹəχ (cléireach) « sacri­stain », non comme pʹi·əkᵊləχ (cf. § 246) ; ɛbʹrʹᴜ: (oib­riughadh) « tra­vailler », mais ebʹⁱrʹɩ (oibre), gén. de ɔbʷɩrʹ (obair) « travail» ; il arrive qu’on entende αmʹⁱlʹi:ᵊχt (aim­lidheacht) « mala­dresse », à côté de αmʹⁱlʹɩhɩ (aimlithe) « maladroit » (mais aussi αimʹlʹi:ᵊχt).

Sans doute faut-il expliquer de même l’opposition entre : αhɩnʹɩ (aithne) « connais­sance » et αnʹhi:m pour *αhnʹi:m (aith­nighim) « je reconnais ».

De même kʹαhərər (ceathrar) « quatre personnes », mais kʹαrhᴜ: (ceath­ramha) « quart » (ou aussi kʹαhərᴜ:).

Pour les groupes de trois consonnes, cf. § 232.

§ 250. 
rəχo:ⁱdʹ (urchóid) « désastre » ; rəχo:ⁱdʲəχ (urchóid­each) « désas­treux » ; rʌχər (urchor) « jet »



In [1933]:
# DATA[section] = []
section = 246
if not section in DATA:
    DATA[section] = []
text = REST[section].replace(" : ", " ; ")
text = text.replace("\u00ad", "")
text = text.replace("»", " »")
text = text.replace("  »", " »")
extract(text, ";", 7)

9


In [1927]:
def split_trans(pos):
    if " ou " in DATA[section][pos]["transcription"]:
        DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" ou ")
    elif " et " in DATA[section][pos]["transcription"]:
        DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" et ")
    elif " à côté de " in DATA[section][pos]["transcription"]:
        DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" à côté de ")

for s in range(len(DATA[section])):
    if "transcription" in DATA[section][s]:
        split_trans(s)

In [1935]:
import json

if str(section) in DATA and section in DATA:
    tmp = DATA[str(section)] + DATA[section]
    DATA[str(section)] = tmp
    del(DATA[section])

with open("sjoestedt_phonetique.json", "w") as outf:
    json.dump(DATA, outf)

In [1934]:
def french_to_note(pos):
    DATA[section][pos]["french_note"] = DATA[section][pos]["french"]
    del(DATA[section][pos]["french"])

def contrast_next(pos):
    if 'irish' in DATA[section][pos+1]:
        DATA[section][pos]['contrast'] = DATA[section][pos+1]['irish']
    else:
        DATA[section][pos]['contrast'] = DATA[section][pos+1]['transcription']
    DATA[section][pos]['contrast_id'] = DATA[section][pos+1]['id']

contrast_next(-2)

In [1502]:
mods = [
    {
      "section": 17,
      "id": "17_23",
      "irish": "lomaim",
      "transcription": "lɔmʷɩmʹ",
      "french": "je me dépouille, je deviens chauve",
      "future": "lomfad"
    },
    {
      "section": 177,
      "id": "177_25",
      "transcription": "sɔnə",
      "irish": "sona",
      "french": "heureux"
    },
    {
      "section": 177,
      "id": "177_23",
      "transcription": "krɔmʷɩmʹ",
      "irish": "cromaim",
      "french": "je courbe"
    },
   {
      "section": 177,
      "id": "177_24",
      "transcription": "fɔnəvər",
      "irish": "fonnmhar",
      "french": "désireux"
    },
]
modwith = ["lo̤mʷɩmʹ", "so̤nə", "kro̤mʷɩmʹ", "fo̤nəvər"]

section = 180
DATA[section] = []
for i in range(len(mods)):
    mod = mods[i]
    mod["section"] = section
    mod["compare_id"] = mod["id"]
    mod["id"] = f"{section}_{i + 1}"
    mod["transcription"] = modwith[i]
    DATA[section].append(mod)
